# Credit Scoring Model Development Pipeline

This notebook walks through the full model development pipeline step-by-step:

1. **Data Loading** - Load parquet, split into Train / Test / OOT
2. **Constant Elimination** - Remove zero-variance features
3. **Missing Elimination** - Remove high missing-rate features
4. **IV Analysis** - Remove low/suspicious Information Value features
5. **PSI Stability** - Remove distribution-unstable features (within training data only)
6. **Correlation Elimination** - Greedy IV-ordered correlation removal
7. **Forward Feature Selection** - XGBoost sequential selection
8. **Model Evaluation** - Train / Test / OOT quarterly performance
9. **Excel Report** - Generate full report

In [ ]:
import sys
from pathlib import Path

# Add project root to path
PROJECT_ROOT = Path().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logging

# Setup logging to see pipeline output
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-5s | %(message)s',
    datefmt='%H:%M:%S',
)

print(f'Project root: {PROJECT_ROOT}')

## Configuration

In [ ]:
# ── Paths ──
INPUT_PATH = str(PROJECT_ROOT / 'data' / 'sample' / 'sample_features.parquet')
OUTPUT_DIR = str(PROJECT_ROOT / 'outputs')

# ── Data split ──
TRAIN_END_DATE = '2024-06-30'   # Everything after this -> OOT quarters
TEST_SIZE = 0.20                 # 20% of training period for test set
TARGET_COLUMN = 'target'
DATE_COLUMN = 'application_date'

# ── Elimination thresholds ──
IV_MIN = 0.02          # Below -> useless
IV_MAX = 0.50          # Above -> suspicious (data leakage risk)
MISSING_THRESHOLD = 0.70
PSI_THRESHOLD = 0.25   # Critical PSI threshold
CORRELATION_THRESHOLD = 0.90
AUC_THRESHOLD = 0.0001 # Minimum AUC improvement for feature selection

---
## 1. Data Loading & Splitting

In [ ]:
from src.model_development.data_loader import load_and_split

datasets = load_and_split(
    input_path=INPUT_PATH,
    train_end_date=TRAIN_END_DATE,
    target_column=TARGET_COLUMN,
    date_column=DATE_COLUMN,
    test_size=TEST_SIZE,
)

features = datasets.feature_columns
X_train = datasets.train[features]
y_train = datasets.train[TARGET_COLUMN]
X_test = datasets.test[features]
y_test = datasets.test[TARGET_COLUMN]

print(f'\nFeature columns: {len(features)}')
print(f'Train: {len(X_train):,} rows, bad rate: {y_train.mean():.2%}')
print(f'Test:  {len(X_test):,} rows, bad rate: {y_test.mean():.2%}')
print(f'OOT quarters: {datasets.oot_labels}')
for label in datasets.oot_labels:
    q = datasets.oot_quarters[label]
    print(f'  {label}: {len(q):,} rows, bad rate: {q[TARGET_COLUMN].mean():.2%}')

In [ ]:
# Quick look at date distribution
fig, ax = plt.subplots(figsize=(12, 4))
dates = datasets.train[DATE_COLUMN]
dates.dt.to_period('M').value_counts().sort_index().plot(kind='bar', ax=ax, color='steelblue')
ax.set_title('Training Data - Monthly Application Counts')
ax.set_xlabel('Month')
ax.set_ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

---
## 2. Constant Feature Elimination

In [ ]:
from src.model_development.eliminators import ConstantEliminator

const_elim = ConstantEliminator()
const_result = const_elim.eliminate(X_train, y_train, features)
features = const_result.kept_features

print(f'Eliminated: {const_result.n_eliminated}')
print(f'Remaining:  {const_result.n_kept}')
const_result.details_df[const_result.details_df['Status'] == 'Eliminated'].head(10)

---
## 3. Missing Value Elimination

In [ ]:
from src.model_development.eliminators import MissingEliminator

missing_elim = MissingEliminator(max_missing_rate=MISSING_THRESHOLD)
missing_result = missing_elim.eliminate(X_train, y_train, features)
features = missing_result.kept_features

print(f'Eliminated: {missing_result.n_eliminated}')
print(f'Remaining:  {missing_result.n_kept}')

# Missing rate distribution
fig, ax = plt.subplots(figsize=(10, 4))
missing_result.details_df['Missing_Rate'].hist(bins=50, ax=ax, color='coral')
ax.axvline(x=MISSING_THRESHOLD, color='red', linestyle='--', label=f'Threshold ({MISSING_THRESHOLD:.0%})')
ax.set_title('Missing Rate Distribution')
ax.set_xlabel('Missing Rate')
ax.legend()
plt.tight_layout()
plt.show()

---
## 4. IV (Information Value) Elimination

In [ ]:
from src.model_development.eliminators import IVEliminator

iv_elim = IVEliminator(min_iv=IV_MIN, max_iv=IV_MAX)
iv_result = iv_elim.eliminate(X_train, y_train, features)
features = iv_result.kept_features

print(f'Eliminated: {iv_result.n_eliminated}')
print(f'Remaining:  {iv_result.n_kept}')

# Extract IV scores for downstream use
iv_scores = {}
for _, row in iv_result.details_df.iterrows():
    if row.get('IV_Score') is not None:
        iv_scores[row['Feature']] = row['IV_Score']

In [ ]:
# IV distribution of kept features
kept_iv = iv_result.details_df[iv_result.details_df['Status'] == 'Kept'].copy()

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Histogram
kept_iv['IV_Score'].hist(bins=30, ax=axes[0], color='seagreen')
axes[0].set_title('IV Distribution (Kept Features)')
axes[0].set_xlabel('IV Score')

# IV category counts
cat_order = ['weak', 'medium', 'strong']
cat_counts = kept_iv['IV_Category'].value_counts().reindex(cat_order).fillna(0)
cat_counts.plot(kind='bar', ax=axes[1], color=['#f0ad4e', '#5cb85c', '#5bc0de'])
axes[1].set_title('IV Categories (Kept Features)')
axes[1].set_ylabel('Count')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

# Top 20 features by IV
print('\nTop 20 features by IV:')
kept_iv.head(20)[['Feature', 'IV_Score', 'IV_Category', 'Univariate_AUC', 'Univariate_KS']]

---
## 5. PSI Stability Elimination

PSI checks run **within training data only** — OOT is reserved purely for evaluation.

### Available PSI Checks

| Check | Description |
|-------|-------------|
| `QuarterlyPSICheck()` | Each quarter vs overall training distribution |
| `ConsecutiveQuarterPSICheck()` | Q1 vs Q2, Q2 vs Q3, ... |
| `YearlyPSICheck()` | Each year vs overall training |
| `HalfSplitPSICheck()` | First half vs second half |
| `DateSplitPSICheck('2024-04-01', label='Pre/Post Apr 2024')` | Custom date split |

In [ ]:
from src.model_development.eliminators import (
    PSIEliminator,
    QuarterlyPSICheck,
    ConsecutiveQuarterPSICheck,
    YearlyPSICheck,
    HalfSplitPSICheck,
    DateSplitPSICheck,
)

# Configure PSI checks
psi_checks = [
    QuarterlyPSICheck(),                                           # Each quarter vs all
    ConsecutiveQuarterPSICheck(),                                   # Q(n) vs Q(n+1)
    HalfSplitPSICheck(),                                           # First half vs second half
    DateSplitPSICheck('2024-04-01', label='Pre/Post Apr 2024'),     # Custom date split
]

psi_elim = PSIEliminator(
    critical_threshold=PSI_THRESHOLD,
    checks=psi_checks,
)

psi_result = psi_elim.eliminate(
    X_train, y_train, features,
    train_dates=datasets.train[DATE_COLUMN],
)
features = psi_result.kept_features

print(f'Eliminated: {psi_result.n_eliminated}')
print(f'Remaining:  {psi_result.n_kept}')

In [ ]:
# PSI summary - top features by max PSI
psi_details = psi_result.details_df.copy()

fig, ax = plt.subplots(figsize=(10, 4))
psi_details['Max_PSI'].hist(bins=50, ax=ax, color='mediumpurple')
ax.axvline(x=PSI_THRESHOLD, color='red', linestyle='--', label=f'Critical ({PSI_THRESHOLD})')
ax.axvline(x=0.10, color='orange', linestyle='--', label='Warning (0.10)')
ax.set_title('Max PSI Distribution Across Features')
ax.set_xlabel('Max PSI')
ax.legend()
plt.tight_layout()
plt.show()

# Show warnings and eliminations
warnings_df = psi_details[psi_details['Max_PSI'] >= 0.10].sort_values('Max_PSI', ascending=False)
if len(warnings_df) > 0:
    print(f'\nFeatures with PSI >= 0.10 (warnings + eliminations): {len(warnings_df)}')
    display(warnings_df[['Feature', 'Max_PSI', 'Mean_PSI', 'Status']].head(20))
else:
    print('\nAll features are stable (Max PSI < 0.10)')

---
## 6. Correlation Elimination

In [ ]:
from src.model_development.eliminators import CorrelationEliminator

corr_elim = CorrelationEliminator(max_correlation=CORRELATION_THRESHOLD)
corr_result = corr_elim.eliminate(X_train, y_train, features, iv_scores=iv_scores)
features = corr_result.kept_features

print(f'Eliminated: {corr_result.n_eliminated}')
print(f'Remaining:  {corr_result.n_kept}')

# Correlated pairs
if len(corr_result.details_df) > 0:
    print(f'\nCorrelated pairs eliminated:')
    display(corr_result.details_df.head(20))

In [ ]:
# Show remaining features sorted by IV
remaining = pd.DataFrame({
    'Feature': features,
    'IV': [iv_scores.get(f, 0) for f in features],
}).sort_values('IV', ascending=False)

print(f'{len(features)} features entering forward selection:\n')
display(remaining)

---
## 7. Forward Feature Selection (XGBoost)

In [ ]:
from src.model_development.feature_selector import forward_feature_selection

selected_features, selection_df, final_model = forward_feature_selection(
    X_train=X_train[features],
    y_train=y_train,
    X_test=X_test[features],
    y_test=y_test,
    features=features,
    iv_scores=iv_scores,
    auc_threshold=AUC_THRESHOLD,
)

print(f'\nSelected features: {len(selected_features)}')
print(f'Final model test AUC: {selection_df[selection_df["Status"]=="Added"]["AUC_After"].iloc[-1]:.6f}')

In [ ]:
# Selection progress
added = selection_df[selection_df['Status'] == 'Added'].copy()

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(added['Cumulative_Features'], added['AUC_After'], 'o-', color='steelblue', markersize=6)
ax.set_xlabel('Number of Features')
ax.set_ylabel('Test AUC')
ax.set_title('Forward Selection: AUC vs Number of Features')
ax.grid(True, alpha=0.3)

# Annotate each point with feature name
for _, row in added.iterrows():
    ax.annotate(
        row['Feature'][:25],
        (row['Cumulative_Features'], row['AUC_After']),
        fontsize=7, rotation=30, ha='left', va='bottom',
    )

plt.tight_layout()
plt.show()

# Full selection table
display(added[['Step', 'Feature', 'Feature_IV', 'AUC_After', 'AUC_Improvement']])

---
## 8. Model Evaluation (Train / Test / OOT)

In [ ]:
from src.model_development.evaluator import evaluate_model_quarterly

performance_df, lift_tables, importance_df = evaluate_model_quarterly(
    model=final_model,
    selected_features=selected_features,
    train_df=datasets.train,
    test_df=datasets.test,
    oot_quarters=datasets.oot_quarters,
    target_column=TARGET_COLUMN,
)

print('Model Performance Summary:')
display(performance_df)

In [ ]:
# AUC & Gini across periods
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

x = range(len(performance_df))
labels = performance_df['Period'].tolist()

# AUC
colors = ['steelblue' if 'OOT' not in l else 'coral' for l in labels]
axes[0].bar(x, performance_df['AUC'], color=colors)
axes[0].set_xticks(x)
axes[0].set_xticklabels(labels, rotation=45, ha='right')
axes[0].set_ylabel('AUC')
axes[0].set_title('AUC Across Periods')
axes[0].set_ylim(0.5, 1.0)
for i, v in enumerate(performance_df['AUC']):
    axes[0].text(i, v + 0.005, f'{v:.4f}', ha='center', fontsize=8)

# Gini
axes[1].bar(x, performance_df['Gini'], color=colors)
axes[1].set_xticks(x)
axes[1].set_xticklabels(labels, rotation=45, ha='right')
axes[1].set_ylabel('Gini')
axes[1].set_title('Gini Across Periods')
axes[1].set_ylim(0.0, 1.0)
for i, v in enumerate(performance_df['Gini']):
    axes[1].text(i, v + 0.01, f'{v:.4f}', ha='center', fontsize=8)

plt.tight_layout()
plt.show()

In [ ]:
# Feature Importance
fig, ax = plt.subplots(figsize=(10, max(4, len(importance_df) * 0.4)))
imp = importance_df.sort_values('Importance', ascending=True)
ax.barh(imp['Feature'], imp['Importance'], color='steelblue')
ax.set_xlabel('Importance')
ax.set_title('Feature Importance (Final Model)')
plt.tight_layout()
plt.show()

In [ ]:
# Lift table for Test set
print('Lift Table - Test Set:')
display(lift_tables.get('Test', pd.DataFrame()))

---
## 9. Generate Excel Report

In [ ]:
from src.model_development import excel_reporter
from datetime import datetime

run_id = datetime.now().strftime('%Y%m%d_%H%M%S')
output_dir = Path(OUTPUT_DIR)
output_dir.mkdir(parents=True, exist_ok=True)
excel_path = str(output_dir / f'model_dev_{run_id}.xlsx')

# Collect all elimination results
elimination_results = [
    const_result,
    missing_result,
    iv_result,
    psi_result,
    corr_result,
]

# Build summary
train_dates = datasets.train[DATE_COLUMN]
oot_labels = ', '.join(datasets.oot_labels) if datasets.oot_labels else 'None'

summary = {
    'Run Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'Run ID': run_id,
    'Input File': INPUT_PATH,
    'Train End Date': TRAIN_END_DATE,
    'Train Period': f"{train_dates.min().strftime('%Y-%m-%d')} to {train_dates.max().strftime('%Y-%m-%d')}",
    'OOT Periods': oot_labels,
    'Train Rows': len(datasets.train),
    'Test Rows': len(datasets.test),
    'Train Bad Rate': f"{datasets.train[TARGET_COLUMN].mean():.2%}",
    'Test Bad Rate': f"{datasets.test[TARGET_COLUMN].mean():.2%}",
    '': '',
    'Total Features': len(datasets.feature_columns),
    'After Constant Elimination': f"{const_result.n_kept} ({const_result.n_eliminated} eliminated)",
    'After Missing Elimination': f"{missing_result.n_kept} ({missing_result.n_eliminated} eliminated)",
    'After IV Elimination': f"{iv_result.n_kept} ({iv_result.n_eliminated} eliminated)",
    'After PSI Elimination': f"{psi_result.n_kept} ({psi_result.n_eliminated} eliminated)",
    'After Correlation Elimination': f"{corr_result.n_kept} ({corr_result.n_eliminated} eliminated)",
    'After Sequential Selection': f"{len(selected_features)} ({corr_result.n_kept - len(selected_features)} skipped)",
    ' ': '',
    'IV Range': f'[{IV_MIN}, {IV_MAX}]',
    'Missing Threshold': f'{MISSING_THRESHOLD:.0%}',
    'PSI Threshold': str(PSI_THRESHOLD),
    'Correlation Threshold': str(CORRELATION_THRESHOLD),
    'AUC Threshold': str(AUC_THRESHOLD),
}

# Add performance metrics to summary
for _, row in performance_df.iterrows():
    summary[f"AUC {row['Period']}"] = row['AUC']
    summary[f"Gini {row['Period']}"] = row['Gini']

corr_pairs_df = getattr(corr_elim, 'corr_pairs_df', None)

excel_reporter.generate_report(
    output_path=excel_path,
    summary=summary,
    elimination_results=elimination_results,
    corr_pairs_df=corr_pairs_df,
    selection_df=selection_df,
    performance_df=performance_df,
    lift_tables=lift_tables,
    importance_df=importance_df,
)

print(f'Excel report saved to: {excel_path}')

---
## Pipeline Funnel Summary

In [ ]:
# Elimination funnel
steps = ['Total', 'Constant', 'Missing', 'IV', 'PSI', 'Correlation', 'Selection']
counts = [
    len(datasets.feature_columns),
    const_result.n_kept,
    missing_result.n_kept,
    iv_result.n_kept,
    psi_result.n_kept,
    corr_result.n_kept,
    len(selected_features),
]

fig, ax = plt.subplots(figsize=(10, 5))
bars = ax.bar(steps, counts, color=['#2c3e50'] + ['#e74c3c']*5 + ['#27ae60'])
bars[0].set_color('#2c3e50')
bars[-1].set_color('#27ae60')

for bar, count in zip(bars, counts):
    ax.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 5,
            str(count), ha='center', va='bottom', fontweight='bold')

ax.set_ylabel('Number of Features')
ax.set_title('Feature Elimination Funnel')
plt.tight_layout()
plt.show()

print(f'Started with {counts[0]} features, ended with {counts[-1]} selected features.')

---
## Alternative: Run Full Pipeline in One Call

Instead of running each step manually, you can use the `ModelDevelopmentPipeline` class.

In [ ]:
# from src.model_development.pipeline import ModelDevelopmentPipeline
#
# pipeline = ModelDevelopmentPipeline(
#     input_path=INPUT_PATH,
#     train_end_date=TRAIN_END_DATE,
#     output_dir=OUTPUT_DIR,
#     iv_min=IV_MIN,
#     iv_max=IV_MAX,
#     missing_threshold=MISSING_THRESHOLD,
#     psi_threshold=PSI_THRESHOLD,
#     correlation_threshold=CORRELATION_THRESHOLD,
#     auc_threshold=AUC_THRESHOLD,
#     test_size=TEST_SIZE,
#     psi_checks=[
#         QuarterlyPSICheck(),
#         ConsecutiveQuarterPSICheck(),
#         HalfSplitPSICheck(),
#         DateSplitPSICheck('2024-04-01', label='Pre/Post Apr 2024'),
#     ],
# )
#
# results = pipeline.run()
# print(f"Selected features: {results['after_selection']}")
# print(f"Excel report: {results['excel_path']}")